In [1]:
# AOC - Day14 - Part 1 attempt: 2
# using testdata-03

In [2]:
import math

In [3]:
class Chemical:
    # need to do something special when this chemical is made
    # this is the ore, which gets mined, not made
    global o_str
    global ch
    def __init__(self, name, qty, rqty, reactants):
        self.name = name
        self.qty = qty
        self.rqty = rqty
        self.reactants = reactants
        return
    def make(self, req_qty):
        global ore_qty
        # if ore is being requested
        print("{}.make({})...".format(self.name, req_qty))
        if(self.name == o_str):
            # add this amount to accumulator
            ore_qty = ore_qty + req_qty
            # and return what is requested (instantly mined!)
            print("{} ORE mined! total so far: {}".format(req_qty, ore_qty))
            return(req_qty)
        # so ore not being requested.
        # see what is in inventory, and only kick off the reaction for what inventory can't provide
        need_amt = req_qty - self.qty
        if(need_amt <= 0):
            # have enough on hand.  reduce the inventory and return the requested amount.
            print("...enough on hand: {}".format(self.qty))
            self.qty -= req_qty
            return(req_qty)
        else:
            # determine factor required given need_amt and unit_quantity in the make reaction
            factor = calc_factor(need_amt, self.rqty)
            # now 'do the reaction' - iterate through the reactants
            for r in self.reactants:
                # get the ch array element for this reactant
                c = find_chem_by_name(ch, symnum(r)[0])
                req_amt = symnum(r)[1] * factor
                print("{} reactant: {} unit: {}, factor: {}, req_amt: {}, qty on-hand before make: {}".format(self.name, symnum(r)[0], symnum(r)[1], factor, req_amt, c.qty))
                made_amt = c.make(req_amt)
                # update on-hand leftover (req_amt is consumed in the calling reaction)
                c.qty = c.qty + made_amt - req_amt
                print("{} reactant: {} made_amt: {}, qty on-hand after make: {}".format(self.name, symnum(r)[0], made_amt, c.qty))
        return(factor*self.rqty)

In [4]:
class Reactant:
    def _init__(self, chem, qty):
        self.chem = chem
        self.qty = qty
        return

class Reaction:
    def __init__(self, reaction_tup):
        self.reactant_list = reaction_tup[0]
        self.product = reaction_tup[1]
        return

In [5]:
def parse_reaction(r_txt):
    # r_txt comes in like: "44 XJWVT, 5 KHKGT, 1 QDVJ, 29 NZVS, 9 GPVTF, 48 HKGWZ => 1 FUEL"
    # parse this out and return a reactant list (r_list) and product (p)
    # each looks like  {'CHEM_SYMBOL': NUMBER}
    r_list = []
    p = {}
    left, right = r_txt.split(" => ")
    p_num, p_chem = right.split()
    p[p_chem] = int(p_num)
    for l in left.split(", "):
        reactant = {}
        reactant[l.split()[1]] = int(l.split()[0])
        r_list.append(reactant)
    return(r_list, p)

In [6]:
def calc_factor(req, makeunit):
    return(int(math.ceil(req / makeunit)))

In [7]:
# this function returns the first key, value of a dictionary as a tuple
def symnum(r):
    sym = list(r.keys())[0]
    num = list(r.values())[0]
    return((sym, num))

In [8]:
def find_chem_by_name(ch, name):
    c = [chem for chem in ch if chem.name == name]
    return(c[0])

In [9]:
# array of chemicals
ch = []
ore_qty = 0
# these are the strings used for the chemical symbols for Fuel and Ore
f_str = "FUEL"
o_str = "ORE"

In [10]:
# read the reactions list from the input file
ifn = "day14-input-test-03.txt"
# parse each line
# each line defines a chemical and contains the reactancts
with open(ifn) as f:
    for line in f:
        r_list, p = parse_reaction(line)
        # create a chemical for the product and add it to the chemical array
        # initial quantity is 0, of course
        ch.append(Chemical(symnum(p)[0], 0, symnum(p)[1], r_list))
# add ore as a reaction (ore => ore)
o_reaction_line = "1 {} => 1 {}".format(o_str, o_str)
r_list, p = parse_reaction(o_reaction_line)
ch.append(Chemical(symnum(p)[0], 0, symnum(p)[1], r_list))

In [11]:
# this will return the chemical element that is the fuel
x = find_chem_by_name(ch, f_str)

In [12]:
# this will invoke the making of 1 unit of fuel
a = x.make(1)

FUEL.make(1)...
FUEL reactant: BHXH unit: 6, factor: 1, req_amt: 6, qty on-hand before make: 0
BHXH.make(6)...
BHXH reactant: ORE unit: 114, factor: 2, req_amt: 228, qty on-hand before make: 0
ORE.make(228)...
228 ORE mined! total so far: 228
BHXH reactant: ORE made_amt: 228, qty on-hand after make: 0
FUEL reactant: BHXH made_amt: 8, qty on-hand after make: 2
FUEL reactant: KTJDG unit: 18, factor: 1, req_amt: 18, qty on-hand before make: 0
KTJDG.make(18)...
KTJDG reactant: ORE unit: 189, factor: 2, req_amt: 378, qty on-hand before make: 0
ORE.make(378)...
378 ORE mined! total so far: 606
KTJDG reactant: ORE made_amt: 378, qty on-hand after make: 0
FUEL reactant: KTJDG made_amt: 18, qty on-hand after make: 0
FUEL reactant: WPTQ unit: 12, factor: 1, req_amt: 12, qty on-hand before make: 0
WPTQ.make(12)...
WPTQ reactant: BMBT unit: 5, factor: 3, req_amt: 15, qty on-hand before make: 0
BMBT.make(15)...
BMBT reactant: VRPVC unit: 14, factor: 3, req_amt: 42, qty on-hand before make: 0
VRPVC.

In [13]:
print("the amount of ore to create 1 fuel is: {}".format(ore_qty))

the amount of ore to create 1 fuel is: 2210736
